# Build relation between entity

In previous tutorial, we have ingested individual entity. Now we want to ingest relation between entities(e.g. lineage, etc.) 

## 1. Prepare the connexion

In [1]:
from metadata.ingestion.ometa.ometa_api import OpenMetadata
from metadata.generated.schema.entity.services.connections.metadata.openMetadataConnection import (OpenMetadataConnection, AuthProvider)
from metadata.generated.schema.security.client.openMetadataJWTClientConfig import OpenMetadataJWTClientConfig

In [2]:
from creds import om_admin_token
server_config = OpenMetadataConnection(
    hostPort="http://datacatalog.casd.local/api",
    authProvider=AuthProvider.openmetadata,
    securityConfig=OpenMetadataJWTClientConfig(
        jwtToken=om_admin_token,
    ),
)
metadata = OpenMetadata(server_config)

In [3]:
# if it returns true, it means the connection is success 
metadata.health_check()

True

## 2. Create the required entities

To illustrate the table level and column level lineage, we need to create some table entities first.

### 2.1 Create a db service

In [4]:
from metadata.generated.schema.api.services.createDatabaseService import CreateDatabaseServiceRequest
from metadata.generated.schema.entity.services.connections.database.common.basicAuth import BasicAuth
from metadata.generated.schema.entity.services.connections.database.mysqlConnection import MysqlConnection
from metadata.generated.schema.entity.services.databaseService import (DatabaseConnection, DatabaseService, DatabaseServiceType,)

db_service = CreateDatabaseServiceRequest(
    name="test-db-service",
    serviceType=DatabaseServiceType.Mysql,
    connection=DatabaseConnection(
        config=MysqlConnection(
            username="db_login",
            authType=BasicAuth(password="db_name"),
            hostPort="http://db_url:1234",
        )
    ),
)

# when we create an entity by using function `create_or_update`, it returns the created instance of the query
db_service_entity = metadata.create_or_update(data=db_service)

## 2.2 Creating a Database 

In [5]:
from metadata.generated.schema.api.data.createDatabase import CreateDatabaseRequest

db_entity_req = CreateDatabaseRequest(
    name="test-db",
    service=db_service_entity.fullyQualifiedName,
)

db_entity = metadata.create_or_update(data=db_entity_req)

## 2.3 Creating the Schema

In [6]:
from metadata.generated.schema.api.data.createDatabaseSchema import CreateDatabaseSchemaRequest

create_schema_req = CreateDatabaseSchemaRequest(
    name="test-schema", 
    database=db_entity.fullyQualifiedName)

# the create request will return the fqn(fully qualified name) of the created schema
schema_entity = metadata.create_or_update(data=create_schema_req)

## 2.4 Creating the Tables

In [7]:
from metadata.generated.schema.api.data.createTable import CreateTableRequest
from metadata.generated.schema.entity.data.table import Column, DataType

table_a = CreateTableRequest(
    name="test_user",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="id", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="age", dataType=DataType.INT,description="age of the user")],
)

table_b = CreateTableRequest(
    name="test_order",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="id", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="product_id", dataType=DataType.BIGINT,description="product id"),
             Column(name="uid", dataType=DataType.BIGINT,description="id of the user which start the order"),],
)

table_c = CreateTableRequest(
    name="user_order_cube",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="uid", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="oid", dataType=DataType.BIGINT,description="id of the order"),
             Column(name="product_id", dataType=DataType.BIGINT,description="product id"),
             Column(name="age", dataType=DataType.BIGINT,description="age of the user"),],
)

table_a_entity = metadata.create_or_update(data=table_a)
table_b_entity = metadata.create_or_update(data=table_b)
table_c_entity = metadata.create_or_update(data=table_c)

## 3. Adding Lineage

With everything prepared, we can now create the Lineage between both Entities. An `AddLineageRequest` type represents the `edge` between two `Entities`, typed under `EntitiesEdge`.

In the below example, we created an edge between table A and B. And the edge is directed with the keyword `fromEntity` and `toEntity`

In our scenario, user use `test_order` and `test_user` to create a new table `user_order_cube`
In below example, we create two lineage: 
- lineage between table `test_order` and `user_order_cube`
- lineage between table `test_user` and `user_order_cube`

In [8]:
from metadata.generated.schema.api.lineage.addLineage import AddLineageRequest
from metadata.generated.schema.type.entityLineage import EntitiesEdge 
from metadata.generated.schema.type.entityReference import EntityReference

add_lineage_request1 = AddLineageRequest(
    edge=EntitiesEdge(
        description="user order cube build lineage",
        fromEntity=EntityReference(id=table_b_entity.id, type="table"),
        toEntity=EntityReference(id=table_c_entity.id, type="table"),
    ),
)

add_lineage_request2 = AddLineageRequest(
    edge=EntitiesEdge(
        description="user order cube build lineage",
        fromEntity=EntityReference(id=table_a_entity.id, type="table"),
        toEntity=EntityReference(id=table_c_entity.id, type="table"),
    ),
)

created_lineage1 = metadata.add_lineage(data=add_lineage_request1)
created_lineage2 = metadata.add_lineage(data=add_lineage_request2)

### 3.1 Fetching Lineage

Finally, let's fetch the lineage from the other node involved:

In [23]:
from metadata.generated.schema.entity.data.table import Table

metadata.get_lineage_by_name(
    entity=Table,
    fqn="test-db-service.test-db.test-schema.test_user",
    # Tune this to control how far in the lineage graph to go
    up_depth=1,
    down_depth=1
)

{'entity': {'id': '07824d18-8a75-4a98-83c8-b6127c3a6c16',
  'type': 'table',
  'name': 'test_user',
  'fullyQualifiedName': 'test-db-service.test-db.test-schema.test_user',
  'displayName': 'test_user',
  'deleted': False,
  'href': 'http://datacatalog.casd.local/api/v1/tables/07824d18-8a75-4a98-83c8-b6127c3a6c16'},
 'nodes': [{'id': '52347ee2-4593-4931-8e6a-7a6f9158de9e',
   'type': 'table',
   'name': 'user_order_cube',
   'fullyQualifiedName': 'test-db-service.test-db.test-schema.user_order_cube',
   'displayName': 'user_order_cube',
   'deleted': False,
   'href': 'http://datacatalog.casd.local/api/v1/tables/52347ee2-4593-4931-8e6a-7a6f9158de9e'}],
 'upstreamEdges': [],
 'downstreamEdges': [{'fromEntity': '07824d18-8a75-4a98-83c8-b6127c3a6c16',
   'toEntity': '52347ee2-4593-4931-8e6a-7a6f9158de9e'}]}

### 3.2 Lineage Details

Note how when adding lineage information we give to the API an [AddLineage](https://github.com/open-metadata/OpenMetadata/blob/main/openmetadata-spec/src/main/resources/json/schema/api/lineage/addLineage.json) Request. This is composed of an Entity Edge, whose definition you can find [here](https://github.com/open-metadata/OpenMetadata/blob/main/openmetadata-spec/src/main/resources/json/schema/type/entityLineage.json).

In a nutshell, an Entity Edge has:

1. The Entity Reference as the lineage origin,
2. The Entity Reference as the lineage destination,
3. Optionally, Lineage Details.

In the Lineage Details property we can pass further information specific about Table to Table lineage:
- `sqlQuery` specifying the transformation,
- An array of `columnsLineage` as an object with an array of source and destination columns, as well as their own specific transformation function,
- Optionally, the Entity Reference of a Pipeline powering the transformation from Table A to Table B.

The API call will be exactly the same as before, but now we will add more ingredients when defining our objects. Let's see how to do that and play with the possible combinations:

First, import the required classes and create a new table:

In [9]:
from metadata.generated.schema.type.entityLineage import (
    ColumnLineage,
    EntitiesEdge,
    LineageDetails,
)

# Prepare a new table for the column lineage
table_d = CreateTableRequest(
    name="user_order_cube_detail",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="uid", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="oid", dataType=DataType.BIGINT,description="id of the order"),
             Column(name="product_id", dataType=DataType.BIGINT,description="product id"),
             Column(name="age", dataType=DataType.BIGINT,description="age of the user"),],
)

table_d_entity = metadata.create_or_update(data=table_d)

####  3.2.1 Column Level Lineage

We can start by linking our columns together. For that we are going to create:

- A `ColumnLineage` object, linking our Table A column ID -> Table C column ID. Note that this can be a list!
- A `LineageDetails` object, passing the column lineage and the SQL query that powers the transformation.

In [10]:
# a column lineage object has two arguments, fromColumns indicates the source columns, toColumn indicates the destination columns
column_uid_lineage1 = ColumnLineage(
    fromColumns=["test-db-service.test-db.test-schema.test_user.id",
                 ],
    toColumn="test-db-service.test-db.test-schema.user_order_cube_detail.uid"
)

column_uid_lineage2 = ColumnLineage(
    fromColumns=["test-db-service.test-db.test-schema.test_order.uid",
                 ],
    toColumn="test-db-service.test-db.test-schema.user_order_cube_detail.uid"
)

column_oid_lineage = ColumnLineage(
    fromColumns=["test-db-service.test-db.test-schema.test_order.id"],
    toColumn="test-db-service.test-db.test-schema.user_order_cube_detail.oid"
)

column_pid_lineage = ColumnLineage(
    fromColumns=["test-db-service.test-db.test-schema.test_order.product_id"],
    toColumn="test-db-service.test-db.test-schema.user_order_cube_detail.product_id"
)

column_age_lineage = ColumnLineage(
    fromColumns=["test-db-service.test-db.test-schema.test_user.age"],
    toColumn="test-db-service.test-db.test-schema.user_order_cube_detail.age"
)

query_detail="""Create TABLE user_order_cube_detail AS
              SELECT test_user.id as uid, 
             test_order.oid, 
             test_order.product_id
             test_user.age
             FROM test_user 
             INNER JOIN test_order 
             ON test_user.id=test_order.uid;"""
             
# a lineage details contains the sql query which does the data transformation
# columnLineage contains the info of column relations
lineage_details1 = LineageDetails(
    sqlQuery=query_detail,
    columnsLineage=[column_uid_lineage1,column_age_lineage],
)

lineage_details2 = LineageDetails(
    sqlQuery=query_detail,
    columnsLineage=[column_uid_lineage2,column_oid_lineage,column_pid_lineage],
)

add_lineage_request1 = AddLineageRequest(
    edge=EntitiesEdge(
        fromEntity=EntityReference(id=table_a_entity.id, type="table"),
        toEntity=EntityReference(id=table_d_entity.id, type="table"),
        lineageDetails=lineage_details1,
    ),
)

add_lineage_request2 = AddLineageRequest(
    edge=EntitiesEdge(
        fromEntity=EntityReference(id=table_b_entity.id, type="table"),
        toEntity=EntityReference(id=table_d_entity.id, type="table"),
        lineageDetails=lineage_details2,
    ),
)
# for one lineage, we can only have two tables, one source, one destination
created_lineage1 = metadata.add_lineage(data=add_lineage_request1)

created_lineage2 = metadata.add_lineage(data=add_lineage_request2)

After running the above command, you could see the `column lineage in the web UI`. 
> Click on the `Lineage` tab, on bottom left corner, you can find a `Layers` button, click on it and choose `column` options.

Another important point, for one lineage, we can only have two tables, one source, one destination. As the table user_order_cube_detail is the product of inner join of two tables, we must create two lineage entities.


### 3.3 Adding a Pipeline Reference

If the data transformation is done a workflow automation tool(e.g. airflow, etc.). We can as well ingest the `reference of the pipeline` used to create the lineage (e.g., the ETL feeding the tables) into OM.

To prepare this example, we need to start by creating the `Pipeline Entity`. As usual, we have the entity hierarchy, a pipeline must be inside a pipeline service. So we'll need to prepare the `Pipeline Service` first.

In [4]:
from metadata.generated.schema.api.data.createPipeline import CreatePipelineRequest
from metadata.generated.schema.api.services.createPipelineService import (
    CreatePipelineServiceRequest,
)
from metadata.generated.schema.entity.services.pipelineService import (
    PipelineConnection,
    PipelineService,
    PipelineServiceType,
    airflowConnection
)

from metadata.generated.schema.entity.services.connections.pipeline.backendConnection import (
    BackendConnection,
)

pipeline_service = CreatePipelineServiceRequest(
    name="airflow_workflow_manager",
    serviceType=PipelineServiceType.Airflow,
    connection=PipelineConnection(
        config=airflowConnection.AirflowConnection(
            hostPort="http://localhost:8080",
            connection=BackendConnection(),
        ),
    ),
)

pipeline_service_entity = metadata.create_or_update(data=pipeline_service)

create_pipeline = CreatePipelineRequest(
    name="user_order_inner_join_pipeline",
    description="My pipeline description",
    sourceUrl=f"https://meta-ingestion.casd.local/dags/airflow_metadata_extraction/grid",
    concurrency=5,
    pipelineLocation="/opt/airflow/dags/airflow_metadata_extraction.py",
    service=pipeline_service_entity.fullyQualifiedName,
)

pipeline_entity = metadata.create_or_update(data=create_pipeline)

 pipeline_request = CreatePipelineRequest(
            name=self.dag.dag_id,
            description=self.dag.description,
            sourceUrl=f"{clean_uri(self.host_port)}/tree?dag_id={self.dag.dag_id}",
            concurrency=self.dag.max_active_tasks,
            pipelineLocation=self.dag.fileloc,
            startDate=self.dag.start_date.isoformat() if self.dag.start_date else None,
            tasks=self.get_om_tasks(),
            service=pipeline_service.fullyQualifiedName,
        )

With the newly created pipeline service, we can now create a new `LineageDetails` which contains three attributes:
- A `sqlQuery` attribute: which stores the sql query
- A `columnsLineage` attribute, passing the column lineage details
- A `pipeline` attribute, specify the pipeline entity which the lineage uses.

> The pipeline lineage can only contain two tables too, one source, one destination. For our inner join example, we need to creat two pipeline lineage creation query. 

In [12]:
pipeline_lineage_details1 = LineageDetails(
    sqlQuery=query_detail,
    columnsLineage=[column_uid_lineage1,  column_age_lineage],
    pipeline=EntityReference(id=pipeline_entity.id, type="pipeline"),
)

pipeline_lineage_details2 = LineageDetails(
    sqlQuery=query_detail,
    columnsLineage=[column_uid_lineage2, column_oid_lineage, column_pid_lineage],
    pipeline=EntityReference(id=pipeline_entity.id, type="pipeline"),
)

add_pipeline_lineage_request1 = AddLineageRequest(
    edge=EntitiesEdge(
        fromEntity=EntityReference(id=table_a_entity.id, type="table"),
        toEntity=EntityReference(id=table_d_entity.id, type="table"),
        lineageDetails=pipeline_lineage_details1,
        description="show how a lineage works",
    ),
)

add_pipeline_lineage_request2 = AddLineageRequest(
    edge=EntitiesEdge(
        fromEntity=EntityReference(id=table_b_entity.id, type="table"),
        toEntity=EntityReference(id=table_d_entity.id, type="table"),
        lineageDetails=pipeline_lineage_details2,
        description="show how a lineage works",
    ),
)

pipeline_lineage1 = metadata.add_lineage(data=add_pipeline_lineage_request1)
pipeline_lineage2 = metadata.add_lineage(data=add_pipeline_lineage_request2)

### 3.4 Automated SQL lineage

The python sdk also provides a function which can `analyse a sql query and generate the lineage automatically`. Below code shows an example. Let's create a new table `user_order_cube_autogen` which is the result of the inner join on uid between `test_user` and `test_order`

In [14]:
# Prepare a new table `test_user_bis`
table_e = CreateTableRequest(
    name="user_order_cube_autogen",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="uid", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="oid", dataType=DataType.BIGINT,description="id of the order"),
             Column(name="product_id", dataType=DataType.BIGINT,description="product id"),
             Column(name="age", dataType=DataType.BIGINT,description="age of the user"),],
)

table_e_entity = metadata.create_or_update(data=table_e)

In [15]:
lineage_service: DatabaseService = metadata.get_by_name(
    entity=DatabaseService, fqn="test-db-service"
)

query_auto_gen="""insert into user_order_cube_autogen(uid, oid, product_id, age) AS
              SELECT test_user.id as uid, 
             test_order.id as oid, 
             test_order.product_id as product_id,
             test_user.age as age
             FROM test_user 
             INNER JOIN test_order 
             ON test_user.id=test_order.uid;"""

metadata.add_lineage_by_query(
    database_service=lineage_service,
    timeout=200, # timeout in seconds
    sql=query_auto_gen # your sql query
)

With the above command, a new table `user_order_cube_autogen` is created, and the table/column level lineage between `test_user`, `test_order` and `user_order_cube_autogen` are fully created as well.

> Check your sql query if the column level lineage is not created correctly.

## 4. The ingestion with CLI 

The python package `openmetadata-ingestion` which we have installed via `pip install openmetadata-ingestion` offers us an CLI as well, we can use it to ingest data into to OM server without using any python code

The general form is 

```shell
metadata lineage -c path/to/config_yaml.yaml
```

Below is an example of the yaml file which can connect to an OM server and create a new lineage between two tables

```yaml
serviceName: test-service-db-lineage
query: insert into tableD(id, id+1) as select id from tableA
# filePath: test.sql
# parseTimeout: 360 # timeout in seconds
workflowConfig:
  # loggerLevel: DEBUG  # DEBUG, INFO, WARN or ERROR
  openMetadataServerConfig:
    hostPort: <OpenMetadata host and port>
    authProvider: <OpenMetadata auth provider>
```

- **serviceName**: Name of the database service which contains the table involved in query.
- **query**: You can specify the raw sql query within the yaml file itself.
- **filePath**: In case the query is too big then you can also save query in a file and pass the path to the file in this field.
- **parseTimeout**: Timeout for the lineage parsing process.
- **workflowConfig**: The main property here is the openMetadataServerConfig, where you can define the host and security provider of your OpenMetadata installation.

## 5. Cleanup

In [29]:
service_id = metadata.get_by_name(
    entity=DatabaseService, fqn="test-db-service"
).id

metadata.delete(
    entity=DatabaseService,
    entity_id=service_id,
    recursive=True,
    hard_delete=True,
)